In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [36]:
%sendtofile config.txt

connection0  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection1  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection2  DoESLiverpool,decafbad00,10.0.100.1
pinled       2
pinscl       17
pinsda       16

Sent 6 lines (224 bytes) to config.txt.


In [29]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py


Sent 647 lines (23171 bytes) to mqtt_as.py.


In [2]:
%sendtofile main.py

import time, sys, machine, struct
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient
from stdmqttas import pinled, flashpinled, shortmac, topicstem, itertools_count

import network
network.WLAN().active(0)  # disable the connection at startup
flashpinled(5, 300, 300)
print("topicstem", topicstem)

topicplantower = topicstem+"/plantower"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"

uart = machine.UART(2, 9600)

async def plantowertask(client):
    try:
        for n in itertools_count():
            i = 0
            checksum = 0
            while True:
                await asyncio.sleep_ms(10)
                u1 = uart.read(1)
                if u1 == None:
                    await asyncio.sleep_ms(100)
                    continue
                checksum += u1[0]
                if u1[0] == b"BM"[i]:
                    i += 1
                    if i == 2:
                        break
                else:
                    i = 0
                    checksum = 0
            sleng = uart.read(2)
            leng = struct.unpack(">H", sleng)[0]
            checksum += sum(sleng)
            fdata = uart.read(leng)
            data = struct.unpack(">HHHHHHHHHHHHHH", fdata)
            checksum += sum(fdata[:-2])
            if checksum == data[13]:
                print(n, data[:13])
                if client.isconnected():
                    pinled.value(1)
                    await client.publish(topicplantower, "%d %d %d %d %d %d %d %d %d %d %d %d %d"%data[:13])
                    pinled.value(0)
    except OSError as e:
        print("plantowertask", e)
        flashpinled(5, 300, 100)
        aloop.create_task(plantowertask(client))

        
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(plantowertask(client))
aloop.create_task(mqttconnecttask(client))

aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))


Sent 84 lines (2787 bytes) to main.py.
